# Aircraft Object Detection Notebook

### Import Dependencies

In [ ]:

import os
import wget
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

### 1. Files

In [7]:
#configure these based on model choice (Tensorflow 2 object detection zoo)
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'


In [ ]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt

### 2. Setup Folders and Files


In [10]:
filepaths = {
    'COLLECTIMAGE': os.path.join('Tensorflow','workspace','images','collectedimages'),
    'IMAGE_PATH': os.path.join('Tensorflow','workspace','images'),
    'TRAIN_PATH': os.path.join('Tensorflow','workspace','images','train'),
    'TEST_PATH': os.path.join('Tensorflow','workspace','images','train'),
    'SCRIPTS': os.path.join('Tensorflow','scripts'),
    'FROMAPIMODEL': os.path.join('Tensorflow','models'),
    'OURMODEL': os.path.join('Tensorflow','workspace','models'),
    'PRETRAINMODEL': os.path.join('Tensorflow','workspace','pre-trained-models'),
    'TFJS_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH': os.path.join('Tensorflow','protoc'),
    'ANNOTATION': os.path.join('Tensorflow','workspace','annotations'),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL_NAME)
}
    
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(filepaths['SCRIPTS'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(filepaths['ANNOTATION'], LABEL_MAP_NAME)
}


In [11]:
for path in filepaths.values():
    if not os.path.exists(path):
        !mkdir {path}

In [ ]:
#Populate label_map.pbtxt with labels 

labels = [{'name':'', 'id':1}, {'name':'', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)
!move protoc-3.15.6-win64.zip {filepaths['PROTOC_PATH']}
!cd {filepaths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(filepaths['PROTOC_PATH'], 'bin'))   
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd Tensorflow/models/research/slim && pip install -e . 

In [ ]:
if not os.path.exists(os.path.join(filepaths['FROMAPIMODEL'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {filepaths['FROMAPIMODEL']}

In [ ]:
VERIFICATION_SCRIPT = os.path.join(filepaths['FROMAPIMODEL'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

### 3. Move SoTA model from API into right path

In [ ]:
wget.download(PRETRAINED_MODEL_URL)
!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {filepaths['PRETRAINMODEL']}
!cd {filepaths['PRETRAINMODEL']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

### 4. Convert xml files (labels) to TF records

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(filepaths['TRAIN_PATH'])} -l {files['LABELMAP']} -o {os.path.join(filepaths['ANNOTATION'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(filepaths['TEST_PATH'])} -l {files['LABELMAP']} -o {os.path.join(filepaths['ANNOTATION'], 'test.record')} 

### 5. Moving Model Config file

In [ ]:
#move config file from the standard pretrained model ('template') file to the file for our specific version of the model
!copy {os.path.join(filepaths['PRETRAINMODEL'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(filepaths['CHECKPOINT_PATH'])}

### 6. Updating configuration - Transfer Learning

In [ ]:
#modify configuration
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(filepaths['PRETRAINMODEL'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(filepaths['ANNOTATION'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(filepaths['ANNOTATION'], 'test.record')]

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(filepaths['PRETRAINMODEL'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(filepaths['ANNOTATION'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(filepaths['ANNOTATION'], 'test.record')]

In [3]:
# Adjust these params (generalise)
# Aircraft Non-Aircraft

### 6. Training of Model

In [ ]:
TRAINING_SCRIPT = os.path.join(filepaths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, filepaths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
# run command in cmd to track properly (copy paste)
print(command)

### 7. Evaluate the Model

In [ ]:
command_eval = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, filepaths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], filepaths['CHECKPOINT_PATH'])

In [ ]:
#run command in cmd to track properly (copy paste)
print(command_eval)